In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [14]:
data = pd.read_json('Downloads/politifact_factcheck_data.json', lines=True)
data.head()

,verdict,statement_originator,statement,statement_date,statement_source,factchecker,factcheck_date,factcheck_analysis_link
0,true,Barack Obama,John McCain opposed bankruptcy protections for...,6/11/2008,speech,Adriel Bettelheim,6/16/2008,https://www.politifact.com/factchecks/2008/jun...
1,false,Matt Gaetz,"""Bennie Thompson actively cheer-led riots in t...",6/7/2022,television,Yacob Reyes,6/13/2022,https://www.politifact.com/factchecks/2022/jun...
2,mostly-true,Kelly Ayotte,"Says Maggie Hassan was ""out of state on 30 day...",5/18/2016,news,Clay Wirestone,5/27/2016,https://www.politifact.com/factchecks/2016/may...
3,false,Bloggers,"""BUSTED: CDC Inflated COVID Numbers, Accused o...",2/1/2021,blog,Madison Czopek,2/5/2021,https://www.politifact.com/factchecks/2021/feb...
4,half-true,Bobby Jindal,"""I'm the only (Republican) candidate that has ...",8/30/2015,television,Linda Qiu,8/30/2015,https://www.politifact.com/factchecks/2015/aug...


# Cleaning

In [15]:
data['statement_month'] = pd.to_datetime(data['statement_date']).dt.month
data['statement_year'] = pd.to_datetime(data['statement_date']).dt.year
data['factcheck_month'] = pd.to_datetime(data['factcheck_date']).dt.month
data['factcheck_year'] = pd.to_datetime(data['factcheck_date']).dt.year
data = data.drop(['statement_date', 'factcheck_date'], axis = 1)

In [18]:
data['verdict'].value_counts()

false           5625
half-true       3597
mostly-false    3432
mostly-true     3332
pants-fire      2703
true            2463
Name: verdict, dtype: int64

In [23]:
def verdict_cleaner(verdict):
    if verdict == "mostly-true":
        return "true"
    elif verdict == "mostly-false":
        return "false"
    elif verdict == "pants-fire":
        return "false"
    elif verdict == "half-true":
        return "true"
    else:
        return verdict
data['verdict'] = data['verdict'].apply(lambda x: verdict_cleaner(x))

In [24]:
data

,verdict,statement_originator,statement,statement_source,factchecker,factcheck_analysis_link,statement_month,statement_year,factcheck_month,factcheck_year
0,true,Barack Obama,John McCain opposed bankruptcy protections for...,speech,Adriel Bettelheim,https://www.politifact.com/factchecks/2008/jun...,6,2008,6,2008
1,false,Matt Gaetz,"""Bennie Thompson actively cheer-led riots in t...",television,Yacob Reyes,https://www.politifact.com/factchecks/2022/jun...,6,2022,6,2022
2,true,Kelly Ayotte,"Says Maggie Hassan was ""out of state on 30 day...",news,Clay Wirestone,https://www.politifact.com/factchecks/2016/may...,5,2016,5,2016
3,false,Bloggers,"""BUSTED: CDC Inflated COVID Numbers, Accused o...",blog,Madison Czopek,https://www.politifact.com/factchecks/2021/feb...,2,2021,2,2021
4,true,Bobby Jindal,"""I'm the only (Republican) candidate that has ...",television,Linda Qiu,https://www.politifact.com/factchecks/2015/aug...,8,2015,8,2015
...,...,...,...,...,...,...,...,...,...,...
21147,false,Donald Trump,Says the large trade deficit with Japan stems ...,speech,Jon Greenberg,https://www.politifact.com/factchecks/2019/aug...,8,2019,8,2019
21148,false,Donald Trump Jr.,"""Tens of thousands"" of people leave New York e...",social_media,Jill Terreri Ramos,https://www.politifact.com/factchecks/2019/nov...,11,2019,11,2019
21149,false,Chris Abele,"""I have fought for our shared values without b...",news,Dave Umhoefer,https://www.politifact.com/factchecks/2011/jan...,1,2011,1,2011
21150,false,Bloggers,"""Germany halts all Covid-19 vaccines, says the...",blog,Ciara O'Rourke,https://www.politifact.com/factchecks/2021/sep...,8,2021,9,2021


In [25]:
features = ['statement_originator', 'statement_source', 'factchecker', 'statement_month', 'statement_year', 'factcheck_month', 'factcheck_year']
X = data[features]
y = data['verdict']